# Crypto Trading Environment
### Overview
A reinforcement learning environment for trading crypto pairs. This environment is targeted at trading a single pair. In future iterations of this environment, we will explore allowing the trading of multiple pairs on the same environment.
### Features
- Allows for features / indicators / additional numerical data in the observation space.
- Memory allows for the current timestep's observation space to have X length of historical data most recent to the current timestep.
- Long-only positions as this is common in the crypto-world.
### References
|Reference|Relevance|
|--|--|
|[OpenAI Gym Base](https://github.com/openai/gym/blob/master/gym/core.py)|The base class for our environment. This interface seems to be a standard in the reinforcement learning space.|
|[AnyTrading Foundation](https://github.com/AminHP/gym-anytrading)|We used AnyTrading's work as a reference point for creating our own environment that's a little more talored to our needs.|

### Import Dependencies

In [74]:
from enum import Enum
import gym
from gym import spaces
from gym.utils import seeding
import numpy as np
import pickle

### Import Data
We use the lab we worked on before for market_trading > parse_market_data to get the latest market information from FrostAura Plutus.

In [5]:
model_file_path = './data/featurized_market_data.p'

with open(model_file_path, 'rb') as fp:
    featurized_market_data = pickle.load(fp)

#### Split Data
- Price Movement Data
  - Time
  - Open
  - Close
  - High
  - Low
  - Volume
- Feature Data
  - Indicators 
  - Features
  - Balances
  - Stakes 

In [125]:
pair_data = featurized_market_data['AAVE_BTC']

# Index on time. This dataset contains open, close, high, low, volume and any numerical features / indicators you like.
price_movement_df = pair_data.set_index("time")

### Action Space
This refers to the decisions / actions that can be applied to the environment. Usually decided by some intelligent system like a neural network or a state vector machine.

In [3]:
class Actions(Enum):
    Hold = 0
    Buy = 1
    Sell = 2

### Define the Environment

In [167]:
class CryptoTradingEnv(gym.Env):
    metadata = {'render.modes': ['human']}
    
    def __init__(self, data, memory_window_size=50, seed=None, trading_fee_percentage=0.001):
        assert data.ndim == 2, 'The price movement & features dataframe can only be an array of 2 dimensions (tabular).'
        assert memory_window_size > 0, 'The memory window should be 1 for no memory (only the most recent 1) or a positive number for a length of historical events to keep.'
        assert data.shape[0] > memory_window_size, 'The provided data has to contain at least as many records as the length of the memory window.'
        assert trading_fee_percentage > 0 and trading_fee_percentage < 1, 'A valid trading fee is required. Usually around 0.003 (0.3%).'
        
        self.seed(seed)
        
        # Persist locals.
        self.data = self.__extend_data_columns__(data.fillna(0))
        self.memory_window_size = memory_window_size
        self.trading_fee_percentage = trading_fee_percentage
        self.memory_shape = (memory_window_size, data.shape[1])
        
        # Define spaces. This can be thought of the input and output of any model we build around this environment.
        self.action_space = spaces.Discrete(len(Actions))
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=self.memory_shape, dtype=np.float32)
        
        self.reset()

    def __extend_data_columns__(self, df):
        # TODO: Append internal state columns like balances to data prior to determining the shape of this.
        return df
    
    # TODO: Next we tackle the step function.
    
    def seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        
        return [seed]

    def reset(self):
        self.done = False
        self.current_window_start_index = 0
        self.current_window_end_index = self.memory_window_size
        self.current_window = self.data[self.current_window_start_index:self.current_window_end_index]
        self.total_reward = 0
        self.total_profit = 1.0

        return self.current_window

In [168]:
env = CryptoTradingEnv(pair_data)

In [169]:
assert env.current_window.shape == env.observation_space.shape